# Cosas por agregar
- Sección de series de tiempo (datetime)

# Import importantes

In [1]:
import kagglehub
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import datetime


# Download latest version
kagglehub.dataset_download("isathyam31/adult-income-prediction-classification")


/home/capitanespiral/Documents/GitHub/data_science/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'/home/capitanespiral/.cache/kagglehub/datasets/isathyam31/adult-income-prediction-classification/versions/1'

# Gathering data

## Datos en general

La función más usada, lejos es `read_csv` de pandas, donde basta que demos el path del archivo y su nombre, y se leerá si no hay mayor problema. Por default, la primera fila se transforma en las columnas del dataframe, que podemos ver usando `dataframe.columns`

In [2]:
path="/home/capitanespiral/Documents/GitHub/data_science"
filename="data.csv"

df=pd.read_csv(f"{path}/{filename}")
print(df)
print(df.columns)

       age          workclass  fnlwgt    education  education-num  \
0       39          State-gov   77516    Bachelors             13   
1       50   Self-emp-not-inc   83311    Bachelors             13   
2       38            Private  215646      HS-grad              9   
3       53            Private  234721         11th              7   
4       28            Private  338409    Bachelors             13   
...    ...                ...     ...          ...            ...   
32556   27            Private  257302   Assoc-acdm             12   
32557   40            Private  154374      HS-grad              9   
32558   58            Private  151910      HS-grad              9   
32559   22            Private  201490      HS-grad              9   
32560   52       Self-emp-inc  287927      HS-grad              9   

            marital-status          occupation    relationship    race  \
0            Never-married        Adm-clerical   Not-in-family   White   
1       Married-civ-spo

---
Opciones más usadas de `read_csv`:
- **sep o delimiter** &rarr; El separador de la data, usualmente se encuentra solo pero a veces es mejor darlo explícitamente. (en el caso que se confunda el identificador automático)
- **header** &rarr; Número de fila que se toma como nombre de columnas y desde el cual comienza la data, por default, cero (inicio del .csv).
- **skiprows** &rarr; Filas a saltar desde el inicio del archivo (indexeando desde cero, se puede entregar lista o tupla)
- **skipfooter** &rarr; Filas a saltar desde el final del archivo.
- **usecols** &rarr; Subset de columnas a usar (secuencia de números o nombres explícitos).

In [7]:
#Si la primera fila está mala, cambiamos el header
df1=pd.read_csv(f"{path}/data_bad_first_row.csv") #Salió pésimo
df2=pd.read_csv(f"{path}/data_bad_first_row.csv",header=1) #Mucho mejor

#Si la segunda está mala
df3=pd.read_csv(f"{path}/data_bad_second_row.csv") #Primer dato pésimo
df4=pd.read_csv(f"{path}/data_bad_second_row.csv",skiprows=[1],header=0) #Nos saltamos la segunda fila y conservamos el header

#Seleccionemos ciertas columnas no más
df5=pd.read_csv(f"{path}/data_bad_first_row.csv",header=1,usecols=[0,3]) #Puede ser con números, aquí la primera y la cuarta
df6=pd.read_csv(f"{path}/data_bad_first_row.csv",header=1,usecols=range(4)) #Puede ser con iteradores, acá me entrega de la primera A la cuarta
df7=pd.read_csv(f"{path}/data_bad_first_row.csv",header=1,usecols=["age","education","workclass"]) #Puede ser con los nombres explícitos

In [8]:
del df1,df2,df3,df4,df5,df6,df7

## Series de Tiempo

Para el caso de series de tiempo, hay que saber trabajar con `Timestamps`, `Datetime`, `Timedelta`, etc...

### Funciones generales 

- **pd.to_datetime()** &rarr; Transforma lo que le entregues en un "Datetime", funciona aceptando varios formatos y si le entregas secuencias.
- **pd.date_range()** &rarr; Para crear puntos en el tiempo equiespaceados, con un "start" un "end", posibilidad de "periods" o "freq"
    - **freq** más comunes:"y" (años),"m" (meses),"W" (semana), "B" (business day), "d" (días), "h" (horas), "min" (minutos), "s" (segundos) Ojo que hay *infinitas* opciones! revisar en https://pandas.pydata.org/docs/user_guide/timeseries.html#dateoffset-objects
- **df.resample().func()** &rarr; Entregando como variable una frecuencia, podemos "resamplear" según la función "func"
- **datetime.min() y .max()** &rarr; Evidente
- **datetimeindex.atributos** &rarr; nos permite acceder a muchas funciones de tiempo como "hour","day","minute","second","dayofyear", "strftime", entre otras (si es una serie, no un índice, agregar dt) 

In [14]:
##datetime con varios formatos
dt1=pd.to_datetime(["13/1/2018", np.datetime64("2018-01-13"), datetime.datetime(2018, 1, 13)],dayfirst=True)
print("dt1:",dt1)

##date_range
#Creando una lista de tiempo cada tres horas
t1="1-1-2000"
t2="2000-3-10"
dt2=pd.date_range(start=t1,end=t2,freq="3h")
print("dt2:",dt2)

#Lo mismo pero con 15 minutos
t1="1-1-2000"
t2="2000-10-3"
dt3=pd.date_range(start=t1,end=t2,freq="15min")
print("dt3:",dt3)

#Si tengo periodos y frecuencia
dt4=pd.date_range("2018-01-01", periods=5, freq="d")
print("dt4:",dt4)

#Como usar el resampleo
idx = pd.date_range("2018-01-01", periods=10, freq="h")
ts = pd.Series(range(len(idx)), index=idx)
print("ts",ts)
ts_resampled=ts.resample("2h").mean()
print("ts_resampled",ts_resampled)

dt1: DatetimeIndex(['2018-01-13', '2018-01-13', '2018-01-13'], dtype='datetime64[ns]', freq=None)
dt2: DatetimeIndex(['2000-01-01 00:00:00', '2000-01-01 03:00:00',
               '2000-01-01 06:00:00', '2000-01-01 09:00:00',
               '2000-01-01 12:00:00', '2000-01-01 15:00:00',
               '2000-01-01 18:00:00', '2000-01-01 21:00:00',
               '2000-01-02 00:00:00', '2000-01-02 03:00:00',
               ...
               '2000-03-08 21:00:00', '2000-03-09 00:00:00',
               '2000-03-09 03:00:00', '2000-03-09 06:00:00',
               '2000-03-09 09:00:00', '2000-03-09 12:00:00',
               '2000-03-09 15:00:00', '2000-03-09 18:00:00',
               '2000-03-09 21:00:00', '2000-03-10 00:00:00'],
              dtype='datetime64[ns]', length=553, freq='3h')
dt3: DatetimeIndex(['2000-01-01 00:00:00', '2000-01-01 00:15:00',
               '2000-01-01 00:30:00', '2000-01-01 00:45:00',
               '2000-01-01 01:00:00', '2000-01-01 01:15:00',
               '20

In [ ]:
df_t=pd.read_csv("biomet1.csv",skiprows=[1],header=0) #Tenemos caso de segunda linea sin sentido
df_t["time_t"]=pd.to_datetime(df_t['date']+" "+df_t['time']) #Lo guardamos en una nueva columna
print(df_t)
print(df_t["time_t"].dt.year)
print(df_t["time_t"].dt.strftime("%Y/%m/%d"))

In [ ]:
#Fijamos un nuevo índice
df_t=df_t.set_index("time_t")
print(df_t.index)
print(df_t.index.day)

In [ ]:
#Ahora cubrimos todo el espacio
ti=df_t.index.min()
tf=df_t.index.max()
time_total=pd.date_range(start=ti,end=tf,freq="30min")
df_t=df_t.reindex(time_total)
print(df_t)

# Links de interés
- Datasets https://www.kaggle.com/datasets
- Documentación de Pandas https://pandas.pydata.org/docs/index.html 
- Cookbook de pandas https://pandas.pydata.org/docs/user_guide/cookbook.html
- Estructura y un par de cosas sobre data engineering https://apmonitor.com/pds/index.php/Main/DataPreparation